# Notebook for fine-tuning a HuggingFace model for Named Entity Recognition through the HuggingFace Trainer-class

Heavily inspired by [this](https://github.com/huggingface/notebooks/blob/master/examples/token_classification.ipynb) notebook.

We will start off by importing necessary packages.

In [30]:
from datasets import load_dataset, load_metric

We then define the model we want to use.

In [31]:
model = "Maltehb/-l-ctra-danish-electra-small-cased"

We will start off by loading the [DaNE](https://www.aclweb.org/anthology/2020.lrec-1.565/) dataset, through the HuggingFace `datasets` function [`load_dataset()`]((https://huggingface.co/docs/datasets/package_reference/loading_methods.html#datasets.load_dataset)).

In [32]:
datasets = load_dataset("dane")

Using custom data configuration default
Reusing dataset dane (C:\Users\z6hjb\.cache\huggingface\datasets\dane\default\0.0.0\5321342ee0a282c6958fe4151e728b84949f0dd70decbf736633c240300a65f6)


The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

Here are both a train, validation and test split with the features: "id, tokens, pos_tags, chunk_tags and ner_tags".

The first sentence from the training data can be seen below.

In [33]:
datasets["train"][0]

{'dep_ids': [2,
  5,
  5,
  5,
  0,
  7,
  5,
  9,
  7,
  11,
  7,
  17,
  17,
  17,
  14,
  15,
  11,
  17,
  22,
  22,
  22,
  18,
  5],
 'dep_labels': [35,
  16,
  28,
  33,
  19,
  35,
  16,
  35,
  18,
  35,
  18,
  1,
  1,
  33,
  22,
  12,
  32,
  11,
  35,
  10,
  30,
  16,
  34],
 'lemmas': ['på',
  'fredag',
  'have',
  'SiD',
  'invitere',
  'til',
  'reception',
  'i',
  'SID-hus',
  'i',
  'anledning',
  'af',
  'at',
  'formand',
  'Kjeld',
  'Christensen',
  'gå',
  'ind',
  'i',
  'den',
  'glad',
  'tresser',
  '.'],
 'morph_tags': ['AdpType=Prep',
  'Definite=Ind|Gender=Com|Number=Sing',
  'Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Act',
  '_',
  'Definite=Ind|Number=Sing|Tense=Past|VerbForm=Part',
  'AdpType=Prep',
  'Definite=Ind|Gender=Com|Number=Sing',
  'AdpType=Prep',
  'Definite=Def|Gender=Neut|Number=Sing',
  'AdpType=Prep',
  'Definite=Ind|Gender=Com|Number=Sing',
  'AdpType=Prep',
  '_',
  'Definite=Def|Gender=Com|Number=Sing',
  '_',
  '_',
  'Mood=Ind|Tense=P

Here it is evident that the tags are already turned into ids where the "actual" labels, in turn, can be seen below.

In [34]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], names_file=None, id=None), length=-1, id=None)

__The following section is a direct copy-paste from the original notebook.__

Since the labels are lists of `ClassLabel`, the actual names of the labels are nested in the `feature` attribute of the object above:

In [35]:
label_list = datasets["train"].features[f"ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [36]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [37]:
show_random_elements(datasets["train"])

,dep_ids,dep_labels,lemmas,morph_tags,ner_tags,pos_tags,sent_id,text,tok_ids,tokens
0,"[6, 6, 2, 5, 6, 0, 6, 6, 10, 6, 6]","[cc, advmod, case, case, nummod, root, nsubj, obj, case, obl, punct]","[men, ikke, førend, i, 1969, gøre, han, nogen, ved, pladeindustri, .]","[_, _, AdpType=Prep, AdpType=Prep, NumType=Card, Mood=Ind|Tense=Past|VerbForm=Fin|Voice=Act, Case=Nom|Gender=Com|Number=Sing|Person=3|PronType=Prs, Gender=Neut|Number=Sing|PronType=Ind, AdpType=Prep, Definite=Def|Gender=Com|Number=Sing, _]","[O, O, O, O, O, O, O, O, O, O, O]","[CCONJ, ADV, ADP, ADP, NUM, VERB, PRON, PRON, ADP, NOUN, PUNCT]",train-v2-912\n,Men ikke førend i 1969 gjorde han noget ved pladeindustrien.\n,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[Men, ikke, førend, i, 1969, gjorde, han, noget, ved, pladeindustrien, .]"
1,"[3, 3, 4, 0, 4, 10, 10, 10, 8, 4, 14, 13, 10, 13, 4]","[det, amod, nsubj, root, punct, mark, nsubj, advmod, fixed, obj, nmod:poss, cc, obj, conj, punct]","[den, demokratisk, ideal, forudsætte, ,, at, vælger, en, bloc, dirigere, historie, og, udvikling, gang, .]","[Gender=Neut|Number=Sing|PronType=Dem, Definite=Def|Degree=Pos|Number=Sing, Definite=Ind|Gender=Neut|Number=Sing, Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Act, _, _, Definite=Def|Gender=Com|Number=Plur, Foreign=Yes, Foreign=Yes, Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Act, Case=Gen|Definite=Def|Gender=Com|Number=Sing, _, Case=Gen|Definite=Def|Gender=Com|Number=Sing, Definite=Ind|Gender=Com|Number=Sing, _]","[O, B-MISC, O, O, O, O, O, O, O, O, O, O, O, O, O]","[DET, ADJ, NOUN, VERB, PUNCT, SCONJ, NOUN, X, X, VERB, NOUN, CCONJ, NOUN, NOUN, PUNCT]",train-v2-204\n,"Det demokratiske ideal forudsætter, at vælgerne en bloc dirigerer historiens og udviklingens gang.\n","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]","[Det, demokratiske, ideal, forudsætter, ,, at, vælgerne, en, bloc, dirigerer, historiens, og, udviklingens, gang, .]"
2,"[2, 0, 2, 7, 7, 7, 2, 2]","[nsubj, root, obj, case, advmod, nummod, obl, punct]","[han, opgøre, prisforskel, til, kun, tre, øre, .]","[Case=Nom|Gender=Com|Number=Sing|Person=3|PronType=Prs, Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Act, Definite=Def|Gender=Com|Number=Sing, AdpType=Prep, _, NumType=Card, Definite=Ind|Gender=Com|Number=Plur, _]","[O, O, O, O, O, O, O, O]","[PRON, VERB, NOUN, ADP, ADV, NUM, NOUN, PUNCT]",train-v2-2253\n,Han opgør prisforskellen til kun tre øre.\n,"[1, 2, 3, 4, 5, 6, 7, 8]","[Han, opgør, prisforskellen, til, kun, tre, øre, .]"
3,"[4, 1, 4, 0, 4, 7, 4, 4]","[nsubj, nummod, aux, root, advmod, case, nummod, punct]","[den, to, have, arbejde, sammen, siden, 1950, .]","[Number=Plur|PronType=Dem, NumType=Card, Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Act, Definite=Ind|Number=Sing|Tense=Past|VerbForm=Part, _, AdpType=Prep, NumType=Card, _]","[O, O, O, O, O, O, O, O]","[PRON, NUM, AUX, VERB, ADV, ADP, NUM, PUNCT]",train-v2-2006\n,De to har arbejdet sammen siden 1950.\n,"[1, 2, 3, 4, 5, 6, 7, 8]","[De, to, har, arbejdet, sammen, siden, 1950, .]"
4,"[0, 1, 4, 2, 1, 7, 1, 7, 7, 11, 9, 1]","[root, obj, case, nmod, obl:loc, cc, conj, obj, xcomp, advmod, advmod, punct]","[hælde, rest, af, piskefløde, i, og, lade, sauce, koge, lidt, ind, .]","[Mood=Imp, Definite=Def|Gender=Com|Number=Sing, AdpType=Prep, Definite=Def|Gender=Com|Number=Sing, _, _, Mood=Imp, Definite=Def|Gender=Com|Number=Sing, VerbForm=Inf|Voice=Act, Degree=Pos, _, _]","[O, O, O, O, O, O, O, O, O, O, O, O]","[VERB, NOUN, ADP, NOUN, ADV, CCONJ, VERB, NOUN, VERB, ADV, ADV, PUNCT]",train-v2-1828\n,Hæld resten af piskefløden i og lad saucen koge lidt ind.\n,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[Hæld, resten, af, piskefløden, i, og, lad, saucen, koge, lidt, ind, .]"
5,"[2, 0, 2, 2, 2, 12, 12, 12, 12, 12, 12, 2, 14, 12, 17, 17, 14, 22, 22, 19, 19, 12, 22, 22, 22, 27, 25, 22, 42, 42, 32, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 22, 44, 42, 42, 47, 45, 49, 42, 2]","[nsubj, root, advmod, advmod, xcomp, punct, cc, nsubj, cop, advmod, det, conj, case, nmod, case, det, obl, punct, advm

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [38]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model, do_lower_case=False, strip_accents = False)

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [39]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

In [40]:
tokenizer("Hej dette er en sætning!")

{'input_ids': [2, 2914, 2390, 1931, 1944, 19054, 5, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [41]:
tokenizer(["Hej", ",", "dette", "er", "en", "sætning", "opdelt", "i", "ord", "."], is_split_into_words=True)

{'input_ids': [2, 2914, 16, 2390, 1931, 1944, 19054, 17270, 77, 2505, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [42]:
example = datasets["train"][4]
print(example["tokens"])

['Hun', 'er', 'selv', 'på', 'femte', 'år', 'lykkeligt', 'gift', 'med', 'sin', 'socialdemokratiske', 'partifælle', ',', 'Mogens', 'Lykketoft', '.']


In [43]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'Hun', 'er', 'selv', 'på', 'femte', 'år', 'lykkeligt', 'gift', 'med', 'sin', 'socialdemokratiske', 'parti', '##f', '##ælle', ',', 'Mogens', 'Lykke', '##to', '##ft', '.', '[SEP]']


Here the words "Zwingmann" and "sheepmeat" have been split in three subtokens.

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can a `[CLS]` and a `[SEP]` above) and then because of those possible splits of words in multiple tokens:

In [44]:
len(example[f"ner_tags"]), len(tokenized_input["input_ids"])

(16, 22)

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [45]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 14, 14, 15, None]


As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [46]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"ner_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

22 22


Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [47]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [48]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [49]:
tokenize_and_align_labels(datasets['train'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[2, 2892, 7344, 1968, 21277, 1035, 15218, 1939, 13297, 1992, 77, 21277, 1035, 17, 6086, 77, 5752, 1937, 1934, 11245, 29800, 10214, 2423, 1999, 77, 1966, 7298, 3071, 6832, 1010, 18, 3], [2, 3767, 3481, 1926, 2204, 13765, 1926, 10668, 77, 2265, 2029, 3094, 4528, 16, 1981, 1944, 3908, 22864, 1940, 18, 3], [2, 3541, 7118, 1926, 4264, 77, 5986, 3832, 30090, 1926, 24415, 1956, 18, 3], [2, 8695, 1940, 10240, 7001, 4490, 2809, 15509, 2201, 1962, 1956, 2852, 17, 16556, 1010, 5911, 1027, 1968, 2161, 3071, 17, 1926, 8075, 8216, 1931, 1987, 3676, 1927, 2552, 2546, 13703, 16, 1934, 1963, 1968, 3137, 23388, 1926, 6424, 16016, 2123, 18, 3], [2, 2555, 1931, 2161, 1955, 15556, 2143, 28528, 4988, 1956, 2429, 22703, 8562, 1049, 3701, 16, 17997, 13706, 7265, 2048, 18, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [50]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at C:\Users\z6hjb\.cache\huggingface\datasets\dane\default\0.0.0\5321342ee0a282c6958fe4151e728b84949f0dd70decbf736633c240300a65f6\cache-9d9cc0e513a632a3.arrow
Loading cached processed dataset at C:\Users\z6hjb\.cache\huggingface\datasets\dane\default\0.0.0\5321342ee0a282c6958fe4151e728b84949f0dd70decbf736633c240300a65f6\cache-bf2d64edcf1ef7fa.arrow
Loading cached processed dataset at C:\Users\z6hjb\.cache\huggingface\datasets\dane\default\0.0.0\5321342ee0a282c6958fe4151e728b84949f0dd70decbf736633c240300a65f6\cache-1341493b0c81719d.arrow


Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [51]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model, num_labels=len(label_list))

Some weights of the model checkpoint at Maltehb/-l-ctra-danish-electra-small-cased were not used when initializing ElectraForTokenClassification: ['generator.embeddings.word_embeddings.weight', 'generator.embeddings.position_embeddings.weight', 'generator.embeddings.token_type_embeddings.weight', 'generator.embeddings.LayerNorm.weight', 'generator.embeddings.LayerNorm.bias', 'generator.encoder.layer.0.attention.self.query.weight', 'generator.encoder.layer.0.attention.self.query.bias', 'generator.encoder.layer.0.attention.self.key.weight', 'generator.encoder.layer.0.attention.self.key.bias', 'generator.encoder.layer.0.attention.self.value.weight', 'generator.encoder.layer.0.attention.self.value.bias', 'generator.encoder.layer.0.attention.output.dense.weight', 'generator.encoder.layer.0.attention.output.dense.bias', 'generator.encoder.layer.0.attention.output.LayerNorm.weight', 'generator.encoder.layer.0.attention.output.LayerNorm.bias', 'generator.encoder.layer.0.intermediate.dense.weig

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [52]:
args = TrainingArguments(
    f"test-ner",
    evaluation_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,
    weight_decay=0.01,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [53]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [54]:
metric = load_metric("seqeval")

This metric takes list of labels for the predictions and references:

In [55]:
labels = [label_list[i] for i in example[f"ner_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'MISC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [56]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [57]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [58]:
trainer.train()

██████▎| 4067/4384 [23:20<01:28,  3.60it/s]
100%|██████████| 4384/4384 [24:50<00:00,  4.11it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

  4%|▍         | 3/71 [00:00<00:02, 23.40it/s]

  6%|▌         | 4/71 [00:00<00:04, 15.38it/s]

  8%|▊         | 6/71 [00:00<00:04, 15.52it/s]

 11%|█▏        | 8/71 [00:00<00:04, 14.66it/s]

 14%|█▍        | 10/71 [00:00<00:03, 15.48it/s]

 17%|█▋        | 12/71 [00:00<00:04, 14.11it/s]

 20%|█▉        | 14/71 [00:00<00:04, 13.16it/s]

 23%|██▎       | 16/71 [00:01<00:03, 14.36it/s]

 25%|██▌       | 18/71 [00:01<00:04, 12.46it/s]

 28%|██▊       | 20/71 [00:01<00:03, 12.89it/s]

 31%|███       | 22/71 [00:01<00:03, 13.05it/s]

 34%|███▍      | 24/71 [00:01<00:03, 13.60it/s]

 37%|███▋      | 26/71 [00:01<00:03, 14.22it/s]

 39%|███▉      | 28/71 [00:01<00:03, 14.21it/s]

 42%|████▏     | 30/71 [00:02<00:03, 13.53it/s]

 45%|████▌     | 32/71 [00:02<00:02, 13.54it/s]

 48%|████▊     | 34/71 [00:02<00:02, 14.45it/s]

 51%|█████     | 36/71 [00:02<00:0

TrainOutput(global_step=4384, training_loss=0.14673101858501017, metrics={'train_runtime': 1496.3621, 'train_samples_per_second': 2.93, 'epoch': 8.0, 'init_mem_cpu_alloc_delta': 2747863, 'init_mem_cpu_peaked_delta': 9669675, 'train_mem_cpu_alloc_delta': 540888, 'train_mem_cpu_peaked_delta': 17594827})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [59]:
trainer.evaluate()


100%|██████████| 71/71 [00:05<00:00, 13.23it/s]


{'eval_loss': 0.09709525853395462,
 'eval_precision': 0.7576656775469832,
 'eval_recall': 0.8080168776371308,
 'eval_f1': 0.7820316488004084,
 'eval_accuracy': 0.9750825341927369,
 'eval_runtime': 5.4442,
 'eval_samples_per_second': 103.596,
 'epoch': 8.0,
 'eval_mem_cpu_alloc_delta': 2371531,
 'eval_mem_cpu_peaked_delta': 1994333}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [63]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

  0%|          | 14/4384 [20:00:19<6244:33:48, 5144.26s/it]

72it [17:05, 306.44s/it]                       
74it [17:06, 214.53s/it]
76it [17:06, 150.19s/it]
78it [17:06, 105.15s/it]
81it [17:06, 73.62s/it] 
83it [17:06, 51.56s/it]
85it [17:06, 36.11s/it]
87it [17:06, 25.29s/it]
89it [17:06, 17.73s/it]
91it [17:07, 12.43s/it]
93it [17:07,  8.72s/it]
95it [17:07,  6.14s/it]
97it [17:07,  4.32s/it]
99it [17:07,  3.05s/it]
101it [17:07,  2.15s/it]
103it [17:08,  1.53s/it]
105it [17:08,  1.09s/it]
107it [17:08,  1.27it/s]
109it [17:08,  1.76it/s]
111it [17:08,  2.38it/s]
113it [17:08,  3.16it/s]
115it [17:08,  4.17it/s]
117it [17:09,  5.38it/s]
119it [17:09,  6.74it/s]
121it [17:09,  8.09it/s]
123it [17:09,  9.19it/s]
125it [17:09, 10.11it/s]
127it [17:09, 11.03it/s]
129it [17:09, 11.72it/s]
131it [17:10, 11.65it/s]
133it [17:10, 10.51it/s]
135it [17:10, 11.90it/s]
137it [17:10, 12.42it/s]
139it [17:10, 13.03it/s]
141it [17:10, 13.39it/s]

{'LOC': {'precision': 0.6651982378854625,
  'recall': 0.8206521739130435,
  'f1': 0.7347931873479319,
  'number': 184},
 'MISC': {'precision': 0.6061643835616438,
  'recall': 0.6082474226804123,
  'f1': 0.6072041166380788,
  'number': 291},
 'ORG': {'precision': 0.7284345047923323,
  'recall': 0.6440677966101694,
  'f1': 0.6836581709145427,
  'number': 354},
 'PER': {'precision': 0.8486842105263158,
  'recall': 0.862876254180602,
  'f1': 0.855721393034826,
  'number': 299},
 'overall_precision': 0.7165492957746479,
 'overall_recall': 0.7216312056737588,
 'overall_f1': 0.7190812720848055,
 'overall_accuracy': 0.9649279118742913}

In [61]:
tokenized_datasets["test"][0]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'input_ids': [2,
  4717,
  5978,
  13627,
  3644,
  9366,
  3622,
  3426,
  1979,
  17172,
  2961,
  17023,
  1926,
  21392,
  1916,
  23255,
  1028,
  27938,
  2570,
  1959,
  16,
  1934,
  8901,
  1996,
  24049,
  2335,
  1944,
  6,
  6221,
  1014,
  6107,
  6,
  18,
  3],
 'labels': [-100,
  0,
  0,
  7,
  0,
  0,
  1,
  1,
  1,
  2,
  2,
  2,
  0,
  1,
  1,
  2,
  2,
  2,
  0,
  0,
  0,
  0,
  5,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -100],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

Don't forget to [update your model](https://huggingface.co/transformers/model_sharing.html) on the [🤗 Model Hub](https://huggingface.co/models). You can then use it only to generate results like the one shown in the first picture of this notebook!

In [62]:
trainer.predict("Hej mit navn er Malte")

AttributeError: 'str' object has no attribute 'keys'

In [75]:
trainer.save_model(".ajjaja")

In [79]:
from pathlib import Path
path = Path.cwd() / "models"
trainer.save_model(path)